#  Problem Statement

Gesture Recognition

You are working as a data scientist at a leading electronics company, ‘SG Electrics' which manufactures state of the art Smart Televisions. As an addition to the smart features of the televisions, you are given the responsibility to develop a model that can recognise 5 different gestures listed below which will help the customers in navigating through the SMART TV without using the remote.

In this project, we are embarking on the development of a sophisticated model designed to proficiently predict the five distinct gestures associated with our smart TV interface. 

This project entails a comprehensive endeavor into gesture recognition technology.

For the implementation of gesture recognition within the context of movie viewing, a curated set of gestures has been meticulously chosen, each with its designated functionality. The selected gestures and their corresponding functionalities are delineated as follows:

Thumbs Up 👍: Increase the volume

Thumbs Down 👎: Lower the volume

Stop ✋: Pause the movie

Left Swipe : Fast Forward 10 seconds

Right Swipe: Fast Backward 10 seconds



In [ ]:
# Downgrading the module scipy to leverage the functionality of the imread and imresize
!pip install scipy==1.1.0

In [ ]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2
import matplotlib.pyplot as plt

We set the random seed so that the results don't vary drastically.

In [ ]:
# Please run this cell twice in order to import the libraries correctly. It errors out in the first try and works fine in the second one.
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
# Mounting google drive to access the images
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
# Importing the files module to upload train and val in order to make them avaialable in CSV format
from google.colab import files

In [ ]:
train = files.upload()

In [ ]:
val = files.upload()

In [ ]:
# Reading the train and val files
train_doc = np.random.permutation(open('/content/train.csv').readlines())
val_doc = np.random.permutation(open('/content/val.csv').readlines())

# Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def get_batch_images(source_path, folder_list, batch_num, batch_size, t, validation, input_shape):
    
    batch_data = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2], input_shape[3])) 
    batch_labels = np.zeros((batch_size,5))
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = np.round(np.linspace(0,29,input_shape[0])).astype(int)

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        dx, dy = np.random.randint(-1.7, 1.8, 2)
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (input_shape[1], input_shape[2]), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
       
    
    if validation:
        batch_data=batch_data
        batch_labels= batch_labels
        
    return batch_data,batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
def generator(source_path, folder_list, batch_size, input_shape, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # iterate over the number of batches
            # yield the batch_data and the batch_labels
            yield get_batch_images(source_path, folder_list, batch, batch_size, t,validation, input_shape)
            

        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield get_batch_images(source_path, folder_list, batch, batch_size, t,validation, input_shape)


In [ ]:
# Function to tweak the batch_size, number of epochs, input_shape and consequently steps_per_epoch and validation steps
def gen(batch_size, epochs, input_shape):
  train_path = '/content/drive/MyDrive/Project_data/train '
  val_path = '/content/drive/MyDrive/Project_data/val'
  num_train_sequences = len(train_doc)
  print('# training sequences =', num_train_sequences)
  num_val_sequences = len(val_doc)
  print('# validation sequences =', num_val_sequences)
  print('batch_size =', batch_size)
  num_epochs = epochs
  print('# epochs =', num_epochs)
  train_generator = generator(train_path, train_doc, batch_size, input_shape)
  val_generator = generator(val_path, val_doc, batch_size, input_shape, validation=True)
  if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
  else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1
  if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
  else:
    validation_steps = (num_val_sequences//batch_size) + 1
  print('# steps per epoch ', steps_per_epoch)
  print('# validation steps ', validation_steps)
  print('Input Shape ', input_shape)
  return (train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs)

# Model Building
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

# Conv3D Models

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
# Defining the structure of model name to be stored
curr_dt_time = datetime.datetime.now()
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# Adding earlystopping callback
earlystop = EarlyStopping(monitor="val_categorical_accuracy", min_delta=0, patience=3, verbose=1)
# Adding model checkpoint
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# Adding reduce LR on plateau
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR, earlystop]

In [ ]:
# Defining a function to plot the training and validation accuracy
def plot(model):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    axes[0].plot(model.history.history['loss'])   
    axes[0].plot(model.history.history['val_loss'])
    axes[0].legend(['loss','val_loss'])

    axes[1].plot(model.history.history['categorical_accuracy'])   
    axes[1].plot(model.history.history['val_categorical_accuracy'])
    axes[1].legend(['categorical_accuracy','val_categorical_accuracy'])

**First we will go ahead and try to build various versions of a Conv3D model which then will be followed by a Conv2D + RNN, also known as LSTM models. We will compare all the models and select the best one in terms of loss, accuracy and number of parameters.** 

## Model 1: Conv3D 
A model considering all the frames in the image and image size being (120,120). BatchNormalization and Pooling layers have also been added.

In [ ]:
# Number of filters, each increasing index refers to the next layer
nb_filters = [8,16,32,64]
# Number of dense nuerons in fully connected layers, the last element refers to the number of classes
nb_dense = [256, 128, 5]

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(30,120,120,3)

# Model Architecture
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))


model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))


model.add(Dense(nb_dense[1], activation='relu'))


#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Defining optimizer 
optimiser = Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 64, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the results
plot(model)

**There are some signs of overfitting as there is a significance difference between categorical accuracy and validation accuracy.**

**Let's do some experiments. Let's try adding dropout layers to the current model to see how it affects our model performance.**

## Model 2: Conv3D 
A model considering all the frames in the image and image size being (120,120). BatchNormalization and Pooling layers, and droput layers have also been added.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(30,120,120,3)

# Model Architecture
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.25))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Defining optimizer
optimiser = Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 64, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the results
plot(model)

Dropout with 0.25 doesn't seem to have dealt with the problem of overfitting.

## Model 3: Conv3D

A model considering only 20 out of the 30 frames in the image and image size being (120,120). BatchNormalization and Pooling layers, and droput layers with a value of 0.50 have also been added. Batch size has been reduced to 20 and number of epochs have been increased to 25.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(20,120,120,3)

# Model Architecture
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.50))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.50))

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.50))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Defining the optimizer
optimiser = Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 20, epochs = 25, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the results
plot(model)

**We observe a considerable amount of overfitting, hence on to the next architecture.**

## Model 4: Conv3D

In this model, we have used all the frames, (120,120) as image shape and reduced the filter size to (2,2,2). BatchNormalization and pooling layers have also been added. No dropouts have been used. Batch size has been increased to 32.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(30,120,120,3)

# Define Architecture
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(2,2,2), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(2,2,2), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,2,2), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,2,2), 
                 padding='same'))
model.add(Activation('relu'))


model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))


model.add(Dense(nb_dense[1], activation='relu'))


#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Define Optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 32, epochs = 25, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the results
plot(model)

**The accuracy has surely increased, but there are still signs of overfitting. We need to experiment further.**

## Model 5: Conv3D

In this model, we have used exactly the same architecture we used in Model 4, but added a dropout layer with a dropout value of 0.25 to address the issue of overfitting.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(30,120,120,3)

# Model Architecture
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(2,2,2), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(2,2,2), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,2,2), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,2,2), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.25))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Define Optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 32, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the model
plot(model)

**This model has given us the best results so far. Let's experiment some more to see if thigs can get better than this one.**

## Model 6: Conv3D

This model is another variation of Model 4. In this model, we have added dropout layers with a dropout value of 0.50. 

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(30,120,120,3)

# Model Architecture
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(2,2,2), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(2,2,2), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,2,2), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,2,2), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.50))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.50))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.50))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Define Optimizer
optimiser = Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 32, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the model
plot(model)

**We haven't seen any appreciable improvements in the results. Let's experiment with some other aspects.**

## Model 7: Conv3D

In an attempt to reduce the number of parameters, we have reduced the number of frames to 16, decreased the batch size to 20 and reduced the size of the filter in the subsequent layers.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape = (16, 120, 120, 3)

model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(2,2,2), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,2,2), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,2,2), 
                 padding='same'))
model.add(Activation('relu'))


model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Defining the optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 20, epochs = 20, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the results
plot(model)

**The performance given by this model isn't up to what we expected. Moving on to the next architecture.**

## Model 8: Conv3D

This model is built by tweeking the dropout value to 0.5 and increasing the filter size back to (3,3,3) in the input layer.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape = (16, 120, 120, 3)

# Model Architecture
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))


model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.50))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.50))


#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Defining optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 64, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the model
plot(model)

**We have been successful in reducing the number of parameters but it has cost us dearly in terms of accuracy.**

## Model 9: Conv3D

This model is another variation of Model 7, with an added dropout layer with a dropout value of 0.25 and set filter size to (3,3,3).

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape = (16, 120, 120, 3)

# Defining Architecture
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))


model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Defining Optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 64, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
plot(model)

**We have seen some improvement in terms of accuracy but the problem of overfitting has still not been solved.**

**We have built a total of 9 Conv3D models by tweeking the parameters like input_shape, kernel_size, dropout layers with different dropout values, batch_size and the number of epochs. Throughout this experimentation journey, we have been able to build several models with acceptable performance out of which we are going to select one.**

# CNN + RNN or LSTM Models

## Model 10: LSTM 

In this model we have used 18 out of the 30 frames along with BatchNormalization, MaxPooling and Dropout layers with a dropout value of 0.25, batch_size of 20 and 20 as the number of epochs.

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, LSTM

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape = (18, 120, 120, 3)

# Model Architecture
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape = input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))


model.add(LSTM(128))
model.add(Dropout(0.25))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
        
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Define Optimizer
optimiser = Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 20, epochs = 20, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the results
plot(model)

**As we can see, there's hardly any overfitting with this one, but we still have a lot of room for improvement in terms of accuracy.**

## Model 11: LSTM 

In this model, we have added another hidden layer to see if it helps in increasing accuracy.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape = (18, 120, 120, 3)

# Model Architecture
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape = input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(512, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))


model.add(TimeDistributed(Flatten()))


model.add(LSTM(128))
model.add(Dropout(0.25))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
        
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Define Optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 20, epochs = 20, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the results
plot(model)

**Addition of another hidden layer did not help us with any increase in the accuracy.**

## Model 12: LSTM

In this model, we have altered the dropout layers by increasing the dropout value to 0.5. Rest of the architecture is same as the one used in Model 10.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape = (18, 120, 120, 3)

# Define Architecture
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape = input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(512, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))


model.add(TimeDistributed(Flatten()))


model.add(LSTM(128))
model.add(Dropout(0.50))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.50))
        
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Define Optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 20, epochs = 20, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the model
plot(model)

**The overall performance of this model is not up to the mark. Neither the accuracy achieved is good nor the problem of overfitting has been addressed.**

## Model 13: LSTM

In this model, we have used all the 30 frames along with an image size of (120,120). BatchNormaliztion, Max Pooling and Dropout layers with a dropout value of 0.50 have also been used. Batch size has been increased to 64 and number of epochs is set to 15.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(30,120,120,3)

# Define Architecture
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape = input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        

model.add(TimeDistributed(Flatten()))


model.add(LSTM(128))
model.add(Dropout(0.50))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.50))
        
model.add(Dense(nb_dense[2], activation='softmax'))


In [ ]:
# Defining Optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 64, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the model
plot(model)                                                                                                                                                                            

**This architecture has also got the problem of overfitting.**

## Model 14: LSTM

In this model, we have tried to experiment with the size of image being fed to the neural network. All the 30 frames are being considered here and the image size has been reduced to (100, 100). BatchNormalization, MaxPooling and Dropout layers with a dropout value of 0.50 have also been used. Batch size is 64 and the number of epochs is set to 15.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(30,100,100,3)

# Define Architecture
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape = input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        

model.add(TimeDistributed(Flatten()))


model.add(LSTM(128))
model.add(Dropout(0.50))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.50))
        
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Defining Optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 64, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the model
plot(model)

**The accuracy has certainly increased but the model is still overfitting on the validation set.**

## Model 15: LSTM

In this model, we have reduced the number of frames to 16 keeping the size of image as (100, 100). Rest all parameters are same as Model 14.

In [ ]:
# Input Shape of the image in the format: (number of frames, image height, image width, number of channels)
input_shape=(16,100,100,3)

# Define Architecture
model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape = input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        

model.add(TimeDistributed(Flatten()))


model.add(LSTM(128))
model.add(Dropout(0.50))
        
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.50))
        
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
# Defining Optimizer
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())

In [ ]:
# Setting batch_size and num_epochs
(train_generator, val_generator, steps_per_epoch, validation_steps, num_epochs) = gen(batch_size = 64, epochs = 15, input_shape = input_shape)

In [ ]:
# Fitting the model
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
# Plotting the results
plot(model)

**The model seems to have achieved a better accuracy but the problem of overfitting is persistent.**

# Best Model

**After building a total of 15 models, the best model which we have selected keeping in mind the number of parameters, the accuracy and the amount of overfitting, is Model 5.**

The optimal model exhibits the following specifications:

1. Training Accuracy: 93%
2. Validation Accuracy: 87.5%
3. Number of Frames: 30
4. Image Size: 120 x 120
5. Filter Size: 2 x 2 x 2
6. Batch Size: 32
7. Number of Epochs: 15
8. Total Parameters: Approximately 840,000